In [1]:
import sys
import os

import dill as pickle

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)

# sys.path.append('./')

# from globals import *

# from data_prep import load_data

# from model import run_model


In [2]:
def model_pipeline(file_name, save_name=None):
    print('*************\n' + file_name)
    if file_name in ['bw', 'ga']:
        measure = 'diff'
    else:
        measure = 'log_ratio'
    if save_name is None:
        save_name = file_name
    print(measure)
    
    if not os.path.exists(os.path.join(OUT_DIR, save_name)):
        os.mkdir(os.path.join(OUT_DIR, save_name))
    
    df, model_cols, mr = run_model(file_name, measure, n_splines=0, 
                                   include_smoking=False, linear=True)
    beta_samples, gamma_samples = LimeTr.sampleSoln(mr.lt, sample_size=100)
    beta_mean = beta_samples.mean(axis=0)
    beta_std = beta_samples.std(axis=0)
    insig_beta = (np.abs(beta_mean) / beta_std) <= 1.645
    reduce_beta = np.abs(beta_mean) < 1e-5
    bad_beta = insig_beta | reduce_beta

    drop_cols = list(np.array(model_cols)[bad_beta[1:]])
    drop_cols = 

    cov_sel_df = pd.DataFrame({
        'covariate':model_cols,
        'beta_mean':beta_mean[1:],
        'beta_std':beta_std[1:],
        'drop':bad_beta[1:]
    })
    cov_sel_df.to_csv(os.path.join(OUT_DIR, save_name, f'{save_name}_cov_selection.csv'), index=False)
    
    n_splines = 20
    if measure == 'log_ratio':
        df_nosmoke, model_cols_nosmoke, mr_nosmoke = run_model(
            file_name, measure, n_splines=n_splines, include_smoking=False  #, drop_cols=drop_cols
        )
        df_nosmoke, model_cols_nosmoke, mr_nosmoke = run_model(
            file_name, measure, n_splines=n_splines, include_smoking=True  #, drop_cols=drop_cols
        )  
    
    n_bins = 100
    pred_df = predict_n_plot(df, model_cols, mr, 
                             n_samples=n_splines*10, n_bins=n_bins, file_name=save_name, measure=measure)
    
    with open(os.path.join(OUT_DIR, save_name, f'{save_name}_mr.pkl'), 'wb') as fopen:
        pickle.dump(mr, fopen, -1)
    
    pred_df.to_csv(os.path.join(OUT_DIR, save_name, f'{save_name}_preds.csv'), index=False)


In [ ]:
if not os.path.exists(OUT_DIR):
    os.mkdir(OUT_DIR)
for file_name in ['cvd_stroke', 'cvd_ihd', 'neo_lung', 't2_dm', 'lri', 'resp_copd', 'bw', 'ga']:
    model_pipeline(file_name)


*************
cvd_stroke
log_ratio
Knot range: 4.859277777777778 to 81.32076117961364
Minimum interval width: 7.6461483401835855
*************
cvd_stroke
log_ratio
Knot range: 7.401327033771469 to 19793.07448965117
Minimum interval width: 1978.56731626174
